In [1]:
# 01: Sample Code for EMSST V1.1
#
# Read all variables from netCDF
# Decode bit_flag for EMS and EMN
# List all data for a specific day and grid point
# 
# Tomita and Hihara (2019)
#------------------------------------------------------------------------------------
import pandas as pd
import netCDF4

In [2]:
# input
#
#  year: four digit year (ex. 2018)
#  jday: julian day of year (1-366)
#  x: grid number for longitude (1-1440)
# y: grid number for latitude (1:720)

year=2018
jday=10
y=360
x=720

In [3]:
#  read variables from netCDF

## open netCDF
file='EMSST_V1.1_DAILY_'+str(year)+'.nc'
ncf = netCDF4.Dataset(file, 'r')

## ret. varaibles
sst = ncf.variables['SST']
ave = ncf.variables['AVE']
std = ncf.variables['STD']
min = ncf.variables['MIN']
max = ncf.variables['MAX']
num = ncf.variables['NUM']
ems = ncf.variables['EMS']
emn = ncf.variables['EMN']

## misc.
jday = jday-1
x = x-1
y = y-1

## convert to integer for bit_flags
iems=int(ems[jday,y,x])
iemn=int(emn[jday,y,x])

## Check 
print('SST =', sst[jday,y,x])
print('AVE =', ave[jday,y,x])
print('STD =', std[jday,y,x])
print('MIN =', min[jday,y,x])
print('MAX =', max[jday,y,x])
print('NUM =', num[jday,y,x])
print('EMS =', iems, '(bit flag)' )
print('EMN =', iemn, '(bit flag)' )

SST = 300.969
AVE = 300.962
STD = 0.47528005
MIN = 300.539
MAX = 301.44998
NUM = 7.0
EMS = 268466 (bit flag)
EMN = 2048 (bit flag)


In [4]:
def get_each_bit(ibit):
    ''' Data Processing Func
    '''
    # 0. convert str
    bit_flag_str = format(ibit, '032b')
    
    # 1. reverse bit_flag
    bit_flag_str_r = bit_flag_str[::-1]
    
    # 2. get each bit and convert into integer
    i=0
    ib=[]
    while i <= 32-1:
        ib.append( int(bit_flag_str_r[i]) )
        i += 1
    return ib


In [5]:
def init():
    product_ids=list(range(0,32))
    product_names=['MGDSST:JMA',
               'OSTIA-NRT:UKMO',
               'AMSR-E:JAXA',
               'AMSR-E:RSS',
               'MW:RSS',
               'OISST:NOAA',
               'OISST+AMSR:NOAA',
               'WSAT:JAXA','WSAT:RSS',
               'TMI','Global Tohoku',
               'AMSR2:JAXA',
               'AMSR2:RSS',
               'GMI:JAXA',
               'GMI:RSS',
               'OSTIA-RA',
               'CMC','GMSSA',
               'MGDSST-RT:JMA',
               20,21,22,23,24,25,26,27,28,29,30,31,32]
    return product_ids, product_names

In [6]:
# Encode bit flag

ids, names = init()
bems = get_each_bit(iems)
bemn = get_each_bit(iemn)

#  make dictionary
data = {'ID':ids, 'Source Product':names, 'EMS (bit_flag)': bems, 'EMN (bit_flag)':bemn}

# make a DataFrame
# # (Note: 19-31 bits are not used in V1.1)
#
df = pd.DataFrame(data)
df = df[:19]


In [7]:
# print the DataFrame for EMS and EMN
# 0: not used
# 1: used
df

,ID,Source Product,EMS (bit_flag),EMN (bit_flag)
0,0,MGDSST:JMA,0,0
1,1,OSTIA-NRT:UKMO,1,0
2,2,AMSR-E:JAXA,0,0
3,3,AMSR-E:RSS,0,0
4,4,MW:RSS,1,0
5,5,OISST:NOAA,1,0
6,6,OISST+AMSR:NOAA,0,0
7,7,WSAT:JAXA,1,0
8,8,WSAT:RSS,0,0
9,9,TMI,0,0
